In [1]:
from scipy.sparse.linalg import svds
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv("/home/marrechea/deep_film/data/ratings_small.csv")
movies = pd.read_csv("/home/marrechea/deep_film/data/movies_metadata.csv")

/home/marrechea/miniconda3/envs/movies/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
movies.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [4]:
movies = movies.drop(columns=['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'video',
       'vote_average', 'vote_count'])

In [5]:
movies.head(3)

,id,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men


In [6]:
movies = movies.rename(columns={"id": "movieId"})
movies.head(3)

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men


In [7]:
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [8]:
ratings.shape

(100004, 4)

In [9]:
ratings = ratings.drop(columns=["timestamp"])

In [10]:
ratings.head(10)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
5,1,1263,2.0
6,1,1287,2.0
7,1,1293,2.0
8,1,1339,3.5
9,1,1343,2.0


In [11]:
ratings.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [12]:
Ratings = ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)
Ratings.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
R = Ratings.values
R

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [14]:
user_ratings_mean = np.mean(R, axis=1)
R_demeaned = R - user_ratings_mean.reshape(-1,1)
R_demeaned

array([[-0.00562541, -0.00562541, -0.00562541, ..., -0.00562541,
        -0.00562541, -0.00562541],
       [-0.02923009, -0.02923009, -0.02923009, ..., -0.02923009,
        -0.02923009, -0.02923009],
       [-0.02007501, -0.02007501, -0.02007501, ..., -0.02007501,
        -0.02007501, -0.02007501],
       ...,
       [-0.01367748, -0.01367748, -0.01367748, ..., -0.01367748,
        -0.01367748, -0.01367748],
       [ 3.98698434, -0.01301566, -0.01301566, ..., -0.01301566,
        -0.01301566, -0.01301566],
       [ 4.95030885, -0.04969115, -0.04969115, ..., -0.04969115,
        -0.04969115, -0.04969115]])

In [15]:
U,sigma,Vt = svds(R_demeaned, k=50)

In [16]:
sigma = np.diag(sigma)

In [17]:
all_user_predicted_ratings = np.dot(np.dot(U,sigma), Vt) + user_ratings_mean.reshape(-1,1)

In [18]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds_df.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.054239,0.045130,-0.004835,-0.019817,-0.011284,0.041373,-0.007822,-0.017188,0.012246,0.037670,...,-0.005258,-0.005453,0.012369,-0.004991,-0.004639,-0.019055,0.021402,-0.006365,-0.006098,-0.004819
1,0.419835,1.406440,-0.188807,0.156658,0.268032,0.414698,0.052172,0.044728,-0.020198,2.220256,...,-0.005909,-0.003974,-0.012555,-0.003555,-0.002711,-0.071621,-0.016212,0.001047,-0.001468,-0.006577
2,1.345619,0.266505,-0.011962,0.012278,0.079508,0.090960,-0.122094,0.031327,-0.018023,0.141176,...,-0.002647,-0.002364,-0.010153,0.000277,-0.000116,-0.018063,-0.015761,0.010611,0.006792,-0.006357
3,1.133455,1.046982,0.141275,0.081841,-0.339675,-1.484659,-0.263096,-0.169750,-0.021862,1.611664,...,0.020805,0.000410,0.056040,-0.002817,-0.000767,0.159159,0.087519,-0.030854,-0.021279,0.048529
4,1.389578,1.466495,0.605557,-0.029647,0.729380,-0.118539,-0.026017,0.065577,-0.156655,0.307926,...,-0.007422,-0.011810,0.006644,-0.005159,-0.001249,-0.034658,0.016456,0.001710,-0.004166,-0.001864


In [19]:
preds_df

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.054239,0.045130,-0.004835,-0.019817,-0.011284,0.041373,-0.007822,-0.017188,0.012246,0.037670,...,-0.005258,-0.005453,0.012369,-0.004991,-0.004639,-0.019055,0.021402,-0.006365,-0.006098,-0.004819
1,0.419835,1.406440,-0.188807,0.156658,0.268032,0.414698,0.052172,0.044728,-0.020198,2.220256,...,-0.005909,-0.003974,-0.012555,-0.003555,-0.002711,-0.071621,-0.016212,0.001047,-0.001468,-0.006577
2,1.345619,0.266505,-0.011962,0.012278,0.079508,0.090960,-0.122094,0.031327,-0.018023,0.141176,...,-0.002647,-0.002364,-0.010153,0.000277,-0.000116,-0.018063,-0.015761,0.010611,0.006792,-0.006357
3,1.133455,1.046982,0.141275,0.081841,-0.339675,-1.484659,-0.263096,-0.169750,-0.021862,1.611664,...,0.020805,0.000410,0.056040,-0.002817,-0.000767,0.159159,0.087519,-0.030854,-0.021279,0.048529
4,1.389578,1.466495,0.605557,-0.029647,0.729380,-0.118539,-0.026017,0.065577,-0.156655,0.307926,...,-0.007422,-0.011810,0.006644,-0.005159,-0.001249,-0.034658,0.016456,0.001710,-0.004166,-0.001864
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,1.445072,0.796437,0.349593,0.118584,0.362366,1.521803,0.537486,-0.001728,0.183798,1.376292,...,-0.004853,-0.006486,-0.001130,-0.001360,0.001742,-0.049968,0.002087,-0.003415,-0.005074,-0.002143
667,0.556714,-0.208781,-0.034636,0.004528,0.010257,0.310522,-0.121078,-0.045900,0.006899,0.004020,...,0.003859,0.001371,-0.002415,0.007073,0.008663,-0.007269,-0.005569,-0.001942,0.000391,0.003827
668,0.728688,-0.135384,0.189810,0.035400,-0.018500,0.242149,0.005227,-0.000577,0.124925,0.030707,...,-0.005106,-0.004248,-0.004701,-0.002367,-0.001945,0.009493,-0.005446,0.001574,-0.000341,-0.007001
669,1.581870,0.084578,-0.046363,-0.013777,0.180043,0.742543,0.004080,-0.009704,-0.016266,0.240628,...,0.005182,0.005733,0.011725,0.006690,0.007145,-0.035748,0.014697,0.007310,0.006698,0.004584


In [32]:
preds_df.T.iloc[:,0].sort_values(ascending=False).index

Int64Index([1374, 2968, 2105, 1954, 2987, 2193, 3175, 3479, 1293, 3671,
            ...
            4226, 5952,  539, 5418, 6711,  357, 4973,  480, 1265,  356],
           dtype='int64', name='movieId', length=9066)

In [36]:
preds_df.T.apply(lambda x:x.sort_values(ascending=False).index).reset_index(drop=True)

,0,1,2,3,4,5,6,7,8,9,...,661,662,663,664,665,666,667,668,669,670
0,1374,296,318,1196,364,4993,260,318,527,1196,...,356,356,293,364,356,590,296,260,318,318
1,2968,590,356,1198,1721,7153,1196,2571,318,260,...,590,2571,2959,588,480,150,318,2858,296,1197
2,2105,150,296,1097,356,5952,1210,5952,2028,1198,...,150,527,5418,4993,593,457,858,1210,527,2571
3,1954,356,527,1240,500,1197,356,4993,608,318,...,457,480,58559,3578,590,296,593,2959,50,4993
4,2987,593,593,1200,595,5349,1198,2959,2571,1210,...,480,593,79132,595,150,608,608,1198,593,4306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9061,357,1356,81834,8784,1210,3,2881,329,4979,56367,...,1374,34405,1028,2288,1278,3147,6365,2161,1258,2366
9062,4973,1210,8368,4973,1198,380,3535,1019,1222,81845,...,1196,2115,1968,1267,1259,2804,2011,47,1278,3168
9063,480,104,5816,5995,541,780,2076,594,344,500,...,1210,1729,150,293,1073,2918,4963,6281,1291,361
9064,1265,1732,40815,62,1196,589,2336,969,3671,46578,...,260,1222,5816,745,1356,2791,2115,4995,1222,51540
